In [204]:
import pandas as pd
import sqlite3
from datetime import datetime

### Criando um dataframe usando o arquivo CSV

In [205]:
path = '~/teste_ailos/base_files/base_originacao.csv'

base_originacao = pd.read_csv(path, sep=';', encoding='latin-1')

In [206]:
base_originacao

,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,13/10/2021,"4.263,60","153,00",36.0
1,A,38,624412,644724,Empréstimo,01/09/2021,"81,60","10,20",10.0
2,A,154,49834,349384,Empréstimo,25/08/2020,"9.761,40","204,00",72.0
3,A,186,398914,444074,Empréstimo,08/01/2021,"4.314,60","163,20",36.0
4,A,430,549582,372569,Empréstimo,02/10/2020,"5.548,80","183,60",48.0
...,...,...,...,...,...,...,...,...,...
7212,E,722959,116706,109717,Financiamento,21/10/2021,"2.131,80","71,40",48.0
7213,E,724099,85550,86794,Empréstimo,24/05/2021,"1.295,40","163,20",9.0
7214,E,726648,4018,68566,Financiamento,13/01/2021,"28.998,60","459,00",120.0
7215,E,727495,4393,92571,Empréstimo,29/06/2021,"7.211,40","265,20",36.0


### Limpando e formatando os dados da tabela

In [207]:
## transformando as colunas numéricas de str para em float 
print(type(base_originacao['VLCONTRATO'][0]))
base_originacao.VLCONTRATO = base_originacao.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
base_originacao.VLPARCELA = base_originacao.VLPARCELA.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))


## transformando a coluna de data em date
base_originacao.DTCONTRATO = pd.to_datetime(base_originacao.DTCONTRATO, format='%d/%m/%Y')

base_originacao.head()

<class 'str'>


/tmp/ipykernel_72875/310650822.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLCONTRATO = base_originacao.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_72875/310650822.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLPARCELA = base_originacao.VLPARCELA.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))


,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,2021-10-13,4263.6,153.0,36.0
1,A,38,624412,644724,Empréstimo,2021-09-01,81.6,10.2,10.0
2,A,154,49834,349384,Empréstimo,2020-08-25,9761.4,204.0,72.0
3,A,186,398914,444074,Empréstimo,2021-01-08,4314.6,163.2,36.0
4,A,430,549582,372569,Empréstimo,2020-10-02,5548.8,183.6,48.0


### Criando um DB Sqlite para fazer consultas em SQL

In [208]:
%reload_ext sql
%sql sqlite:///teste_ailos.db

In [209]:
## criando tabela no db para consulta em SQL
conn = sqlite3.connect('teste_ailos.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS base_originacao')
conn.commit()

base_originacao.to_sql('base_originacao', conn)


7217

# RESOLUÇÃO DO EXERCÍCIO EM SQL

### Calcule a quantidade de dias distintos DTCONTRATO e a soma dos VLCONTRATO dos contratos originados

In [210]:
%%sql
select * from base_originacao limit 5;

   sqlite:///base_originacao.db
 * sqlite:///teste_ailos.db
Done.


index,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,2021-10-13 00:00:00,4263.6,153.0,36.0
1,A,38,624412,644724,Empréstimo,2021-09-01 00:00:00,81.6,10.2,10.0
2,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,204.0,72.0
3,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,163.2,36.0
4,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,183.6,48.0


In [211]:
%%sql
select 
    count(distinct DTCONTRATO) as Qnt_dias 
from
    base_originacao;

   sqlite:///base_originacao.db
 * sqlite:///teste_ailos.db
Done.


Qnt_dias
921


In [212]:
%%sql
select 
    round(sum(VLCONTRATO), 2) as Sum_contrato 
from 
    base_originacao;

   sqlite:///base_originacao.db
 * sqlite:///teste_ailos.db
Done.


Sum_contrato
32816286.6


# RESOLUÇÃO DO EXERCÍCIO EM PYTHON

### Calcule a quantidade de dias distintos DTCONTRATO e a soma dos VLCONTRATO dos contratos originados

In [213]:
base_originacao.head()

,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,2021-10-13,4263.6,153.0,36.0
1,A,38,624412,644724,Empréstimo,2021-09-01,81.6,10.2,10.0
2,A,154,49834,349384,Empréstimo,2020-08-25,9761.4,204.0,72.0
3,A,186,398914,444074,Empréstimo,2021-01-08,4314.6,163.2,36.0
4,A,430,549582,372569,Empréstimo,2020-10-02,5548.8,183.6,48.0


In [214]:
Qnt_dias = base_originacao.DTCONTRATO.unique().shape[0]

print('A quantidade de dias distintos é: ', Qnt_dias)

A quantidade de dias distintos é:  921


In [215]:
Sum_contrato = round(base_originacao.VLCONTRATO.sum(), 2)

print('A soma dos contratos é: ', Sum_contrato)

A soma dos contratos é:  32816286.6
